# Exploratory Analysis
The first step in the analysis of the dataset is a very modest exploratory analysis over the raw dataset. 
This will give us some insights on how the data looks like without any treatment and will allow us to see what variables need some kind of normalization.

In order to do that, we'll be using [SweetViz](https://pypi.org/project/sweetviz/) and [Decision Trees](https://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html#sphx-glr-auto-examples-tree-plot-unveil-tree-structure-py). 

## SweetViz
SweetViz is a Python library that automates the EDA process by creating a multitude of statistical information without any human input. It generates a report on every variable of a dataset.
It's a simple way of finding obvious correlated features.

In [ ]:
import sweetviz
import pandas as pd
df = pd.read_csv('../data/train.csv')

# we're going to analyze all variables in relation to our target, SalePrice
feature_config = sweetviz.FeatureConfig(force_num = ["OverallQual", "OverallCond"])
my_report  = sweetviz.analyze([df,'Train'], target_feat='SalePrice', feat_cfg=feature_config)

# save html
my_report.show_html(filepath="eda_report.html", open_browser=False)

### Report Analysis

#### Target Variable Analysis
Our target is the SalePrice variable: the sale price of the property in dollars. Most of the sales are in the $100k - $200k range (Q1 = $130k, Q3 = $214k), with an average of $181k. The long tail is in the front, with some properties sold for over $300k.

![Target Variable Graph](../media/target_graph.png)

The Pearson correlation chart ranks seven numerical variables as **strongly correlated** to SalePrice:

| Variable      | Pearson |
|--------------	|------	|
| GrLivArea    	| 0.71 	|
| GarageArea   	| 0.62 	|
| TotalBsmtSF  	| 0.61 	|
| 1stFlrSF     	| 0.61 	|
| TotRmsAbvGrd 	| 0.53 	|
| YearBuilt    	| 0.52 	|
| YearRemodAdd 	| 0.51 	|

Those are the usual suspects for real state pricing schemes: Above Grade Area and Number of Rooms (GrLivArea, TotRmsAbvGrd), Garage/Basement/1st Floor Areas (GarageArea, TotalBsmtSF, 1stFlrSF), the age of the building (YearBuilt), and when it was last remodelled (YearRemodAdd).

The Correlation Ratio chart ranks the following categorical variables as **strongly correlated**:

| Variable     	| Correlation 	|
|--------------	|-------------	|
| OverallQual  	| 0.83        	|
| Neighborhood 	| 0.74        	|
| GarageCars   	| 0.70        	|
| ExterQual    	| 0.69        	|
| BsmtQual     	| 0.68        	|
| KitchenQual  	| 0.68        	|
| FullBath     	| 0.58        	|
| GarageFinish 	| 0.55        	|
| FireplaceQu  	| 0.54        	|
| Foundation   	| 0.51        	|

The first variable, **OverallQual**, should have been classified as a Numerical variable. It is a score from 1-10 denoting the quality of the residence build and finish, and should be taken in account together with **OverallCond**. This is a sign that we need to cast this feature to integer manually.

The second variable is **Neighborhood**, containing the district name. This means there is a sale price disparity between neighborhoods, be it because of the neighborhood itself is gentrified or because the houses in that neighborhood are more recent/spacious.

#### Correlated Variable Analysis
Let's go through some of the correlated variables we've discovered in the Target Analysis.

**1. GrLivArea (Area Above Grade)**

![GrLivArea](../media/grlivarea.png)

The *area above grade* is the living area inside a building that is over ground level. We can see that SalePrice rises sharply in tandem with the area, with a few outliers in the far-end of the spectrum. This trend is much more clear if we consider only the values below the 95% threshold (<2466 SF).
Notable associations are **OverallQual** and **Neighborhood**. This means that property quality is connected to the area above grade, and that neighborhoods have different average property sizes.

**2. OverallQual (Overall Quality Score)**

![overallqual](../media/overallqual.png)

The overall quality score is a value between 1 and 10 that denotes the material and the finish of the house. As SweetViz detected this variable as categorical, the first version of this graph wasn't ideal. We forced the cast of OverallQual and OverallCond to numerical in SweetViz and the cell that produces the notebook above has been since updated.

It's a straight correlation between quality and price. Of course: better houses are sold with higher prices. Quality also numerically correlates with the area above grade and age of the building. Interesting categorical associations are **Neighborhood** and **Foundation**, denoting that the neighborhood overall quality and the foundation type influence the overall quality of a property.

**3. Neighborhood (name of the area)**

![neighborhood](../media/neighborhood.png)

The Neighborhood variable is simply the name of the region of the property. It correlates strongly with the property age (YearBuilt, 0.88), sale price (SalePrice, 0.74), and the overall quality (OverallQual, 0.72). Newer properties are usually pricier (with a few historical outliers), and new properties are usually clustered together.

There's a wealth of insights to be gathered from the relationship between Neighborhood and other variables: is the quality of a house inherent to its neighborhood? Was there a recent price spike/drop on houses sold in that region?